# observing-hares-v2
neatened version of notebook `observing-hares.ipynb'

In [2]:
# stock imports
import numpy as np
import pandas as pd
import json
import scipy
import os
import random

##plotting
import matplotlib.pyplot as plt

# plt.style.use('dark_background')
plt.style.use("Solarize_Light2")
plt.rcParams.update({"axes.edgecolor": "black"})
plt.rcParams.update({"text.color": "black"})
plt.rcParams.update({"axes.labelcolor": "black"})
plt.rcParams.update({"xtick.color": "black"})
plt.rcParams.update({"ytick.color": "black"})
plt.rcParams.update({"font.family": "monospace"})

## hares with freq unc
applying observational noise to hares, including perturbing the mode frequencies due to the surface term

In [3]:
def nu_max_range(nu_max_n, mode_min=8, mode_max=13):
    modes = np.random.randint(mode_min, mode_max)
    flip = np.random.randint(2)
    int_half = int(modes * 0.5)
    if flip:
        n_min = nu_max_n - int_half
        n_max = nu_max_n + (modes - int_half)
    else:
        n_min = nu_max_n - (modes - int_half)
        n_max = nu_max_n + int_half

    return n_min, n_max


def obs_noise(true, unc, seed=None):
    seeded_random_state = np.random.RandomState(seed=seed)
    rvs_random_states = seeded_random_state.randint(0, high=2**32 - 1, size=len(true))
    noisy_obs = np.empty(len(true))
    idx = 0
    for ob in true:
        noisy_obs[idx] = scipy.stats.norm(loc=ob, scale=unc[idx]).rvs(
            random_state=rvs_random_states[idx]
        )
        idx += 1

    return noisy_obs


def surf_corr(freqs, nu_max, a, b):
    return freqs + a * ((freqs / nu_max) ** b)


inputs = ["initial_mass", "initial_Zinit", "initial_Yinit", "initial_MLT", "star_age"]

teff_unc = 70  # K
luminosity_unc = 0.04  # L\odot
surface_feh_unc = 0.1  # dex

obs_idx = 0
for hare_idx in range(100):
    hare_df = pd.read_json(f"surf_form/hare{hare_idx}/hare{hare_idx}.json")

    nu_max = hare_df["nu_max"].values[0]
    nu_max_n = hare_df["nu_max_n"].values[0]
    n_min, n_max = nu_max_range(nu_max_n)
    outputs = ["calc_effective_T", "luminosity", "star_feh"] + [
        f"nu_0_{i}" for i in range(n_min, n_max + 1)
    ]

    hare_df = hare_df[inputs + outputs]

    ### add surface correction
    # generate a and b
    a = random.uniform(-10, 2)
    b = random.uniform(4.4, 5.25)

    freqs = hare_df[[f"nu_0_{i}" for i in range(n_min, n_max + 1)]].values[0]

    dnu = np.mean(freqs[1:] - freqs[:-1])

    #nu_max = freqs.mean()
    # shift frequencies
    freqs_corr = surf_corr(freqs, nu_max, a, b)

    # reapply

    hare_cut = hare_df.copy()
    hare_cut.loc[:, [f"nu_0_{i}" for i in range(n_min, n_max + 1)]] = freqs_corr

    frequency_unc = np.random.uniform(0.1, 1)  # \muHz

    obs_unc = np.array(
        [teff_unc, luminosity_unc, surface_feh_unc]
        + [frequency_unc + abs(i - nu_max_n) * 0.1 for i in range(n_min, n_max + 1)]
    )

    hare_obs = obs_noise(hare_cut.drop(inputs, axis=1).values[0], obs_unc)

    hare_obs = obs_noise(hare_cut[outputs].values[0], obs_unc)
    hare_obs_df = hare_cut.copy()
    hare_obs_df[outputs] = hare_obs
    hare_obs_df[["a", "b"]] = [a, b]

    # plt.scatter(hare_obs_df[[f'nu_0_{i}' for i in range(n_min, n_max+1)]]%dnu, hare_obs_df[[f'nu_0_{i}' for i in range(n_min, n_max+1)]], label=f'a={a:.2f}, b={b:.2f}, obs unc')

    plt.xlim((0, dnu))
    # plt.legend()
    fig, ax = plt.subplots()
    ax.scatter(
        np.arange(0, len(obs_unc)), (hare_obs - hare_cut[outputs].values[0]) / obs_unc
    )
    ax.axhline(0, c="black")
    ax.axhline(-1, c="black", linestyle="--")
    ax.axhline(1, c="black", linestyle="--")

    yabs_max = abs(max(ax.get_ylim(), key=abs))
    ax.set_ylim(ymin=-yabs_max, ymax=yabs_max)
    ax.set_xticks(np.arange(0, len(obs_unc)))
    ax.set_xticklabels(outputs)
    # ax.tick_params(axis='x', labelrotation=90)

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    ax.set_title("z-score of observed vs true hare params")
    ax.set_ylabel("z-score")

    path = f"surf_form/hare{hare_idx}/obs{obs_idx}"
    if not os.path.exists(path):
        os.mkdir(path)
        print(f"{path} created!")
    else:
        print(f"{path} already exists", end="\r")
    hare_obs_df.to_json(path + f"/obs{obs_idx}.json")
    pd.DataFrame([obs_unc], columns=outputs).to_json(path + "/uncs.json")
    plt.savefig(path + "/zscore_plot.png", bbox_inches="tight")
    plt.close()
plt.close()

## hares with freq unc
applying observational noise to hares, including perturbing the mode frequencies due to the surface term, but removing the frequency uncertainty to check a and b posteriors

In [3]:
def nu_max_range(nu_max_n, mode_min=8, mode_max=13):
    modes = np.random.randint(mode_min, mode_max)
    flip = np.random.randint(2)
    int_half = int(modes * 0.5)
    if flip:
        n_min = nu_max_n - int_half
        n_max = nu_max_n + (modes - int_half)
    else:
        n_min = nu_max_n - (modes - int_half)
        n_max = nu_max_n + int_half

    return n_min, n_max


def obs_noise(true, unc, seed=None):
    seeded_random_state = np.random.RandomState(seed=seed)
    rvs_random_states = seeded_random_state.randint(0, high=2**32 - 1, size=len(true))
    noisy_obs = np.empty(len(true))
    idx = 0
    for ob in true:
        noisy_obs[idx] = scipy.stats.norm(loc=ob, scale=unc[idx]).rvs(
            random_state=rvs_random_states[idx]
        )
        idx += 1

    return noisy_obs


def surf_corr(freqs, nu_max, a, b):
    return freqs + a * ((freqs / nu_max) ** b)


inputs = ["initial_mass", "initial_Zinit", "initial_Yinit", "initial_MLT", "star_age"]

teff_unc = 70  # K
luminosity_unc = 0.04  # L\odot
surface_feh_unc = 0.1  # dex

obs_idx = 0
for hare_idx in range(100):
    hare_df = pd.read_json(f"no_unc_surf_form/hare{hare_idx}/hare{hare_idx}.json")

    nu_max_n = hare_df["nu_max_n"].values[0]
    nu_max = hare_df["nu_max"].values[0]
    n_min, n_max = nu_max_range(nu_max_n)
    outputs = ["calc_effective_T", "luminosity", "star_feh"] + [
        f"nu_0_{i}" for i in range(n_min, n_max + 1)
    ]

    hare_df = hare_df[inputs + outputs]

    ### add surface correction
    # generate a and b
    a = random.uniform(-10, 2)
    b = random.uniform(4.4, 5.25)

    freqs = hare_df[[f"nu_0_{i}" for i in range(n_min, n_max + 1)]].values[0]

    dnu = np.mean(freqs[1:] - freqs[:-1])

    #nu_max = freqs.mean()
    # shift frequencies
    freqs_corr = surf_corr(freqs, nu_max, a, b)

    # reapply

    hare_cut = hare_df.copy()
    hare_cut.loc[:, [f"nu_0_{i}" for i in range(n_min, n_max + 1)]] = freqs_corr

    frequency_unc = 0

    obs_unc = np.array(
        [teff_unc, luminosity_unc, surface_feh_unc]
        + [frequency_unc + 0.00001 for i in range(n_min, n_max + 1)]
    )

    hare_obs = obs_noise(hare_cut.drop(inputs, axis=1).values[0], obs_unc)

    hare_obs = obs_noise(hare_cut[outputs].values[0], obs_unc)
    hare_obs_df = hare_cut.copy()
    hare_obs_df[outputs] = hare_obs
    hare_obs_df[["a", "b"]] = [a, b]

    # plt.scatter(hare_obs_df[[f'nu_0_{i}' for i in range(n_min, n_max+1)]]%dnu, hare_obs_df[[f'nu_0_{i}' for i in range(n_min, n_max+1)]], label=f'a={a:.2f}, b={b:.2f}, obs unc')

    plt.xlim((0, dnu))
    # plt.legend()
    fig, ax = plt.subplots()
    ax.scatter(
        np.arange(0, len(obs_unc)), (hare_obs - hare_cut[outputs].values[0]) / obs_unc
    )
    ax.axhline(0, c="black")
    ax.axhline(-1, c="black", linestyle="--")
    ax.axhline(1, c="black", linestyle="--")

    yabs_max = abs(max(ax.get_ylim(), key=abs))
    ax.set_ylim(ymin=-yabs_max, ymax=yabs_max)
    ax.set_xticks(np.arange(0, len(obs_unc)))
    ax.set_xticklabels(outputs)
    # ax.tick_params(axis='x', labelrotation=90)

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    ax.set_title("z-score of observed vs true hare params")
    ax.set_ylabel("z-score")

    path = f"no_unc_surf_form/hare{hare_idx}/obs{obs_idx}"
    if not os.path.exists(path):
        os.mkdir(path)
        print(f"{path} created!")
    else:
        print(f"{path} already exists", end="\r")
    hare_obs_df.to_json(path + f"/obs{obs_idx}.json")
    pd.DataFrame([obs_unc], columns=outputs).to_json(path + "/uncs.json")
    plt.savefig(path + "/zscore_plot.png", bbox_inches="tight")
    plt.close()
plt.close()

no_unc_surf_form/hare0/obs0 created!
no_unc_surf_form/hare1/obs0 created!
no_unc_surf_form/hare2/obs0 created!
no_unc_surf_form/hare3/obs0 created!
no_unc_surf_form/hare4/obs0 created!
no_unc_surf_form/hare5/obs0 created!
no_unc_surf_form/hare6/obs0 created!
no_unc_surf_form/hare7/obs0 created!
no_unc_surf_form/hare8/obs0 created!
no_unc_surf_form/hare9/obs0 created!
no_unc_surf_form/hare10/obs0 created!
no_unc_surf_form/hare11/obs0 created!
no_unc_surf_form/hare12/obs0 created!
no_unc_surf_form/hare13/obs0 created!
no_unc_surf_form/hare14/obs0 created!
no_unc_surf_form/hare15/obs0 created!
no_unc_surf_form/hare16/obs0 created!
no_unc_surf_form/hare17/obs0 created!
no_unc_surf_form/hare18/obs0 created!
no_unc_surf_form/hare19/obs0 created!
no_unc_surf_form/hare20/obs0 created!
no_unc_surf_form/hare21/obs0 created!
no_unc_surf_form/hare22/obs0 created!
no_unc_surf_form/hare23/obs0 created!
no_unc_surf_form/hare24/obs0 created!
no_unc_surf_form/hare25/obs0 created!
no_unc_surf_form/hare2